In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import *
from sklearn.metrics import *


In [ ]:
df2019 = pd.read_csv('/content/Cleaned_BRFSS2019 (1).csv')
df2021 = pd.read_csv('/content/Cleaned_BRFSS2021 (1).csv')

In [ ]:
df2019 = df2019[df2019.Diabetes != 1]
df2021 = df2021[df2021.Diabetes != 1]
df2019['Diabetes'] = df2019['Diabetes'].replace(2, 1)
df2021['Diabetes'] = df2021['Diabetes'].replace(2, 1)

<ipython-input-4-f77de3a82621>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2019['Diabetes'] = df2019['Diabetes'].replace(2, 1)
<ipython-input-4-f77de3a82621>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2021['Diabetes'] = df2021['Diabetes'].replace(2, 1)


Train as 2019 and Test/Validation on  2021

In [ ]:
X_train = df2019.drop(columns=['Diabetes'])  # Features
y_train = df2019['Diabetes']  # Target variable
X_test = df2021.drop(columns=['Diabetes'])
y_test = df2021['Diabetes']

In [ ]:
from lightgbm import LGBMClassifier
lgbm_model = LGBMClassifier(random_state = 1016,is_unbalance= True)
lgbm_model.fit(X_train, y_train)
y_pred = lgbm_model.predict(X_test)

lgbm_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", lgbm_accuracy)
print("")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("")
cf_matrix =  confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nAUC Score:\n", roc_auc_score(y_test, y_pred))

print("\nLightGBM:")

lgbm_recall = recall_score(y_test, y_pred)
lgbm_auc = roc_auc_score(y_test, y_pred)
print(f"AUC: {lgbm_auc}")
print(f"Accuracy score: {lgbm_accuracy}")
print(f"Recall score: {lgbm_recall}")
print(f"F1_score: "+str(f1_score(y_test,y_pred)))

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 24494, number of negative: 147245
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 203
[LightGBM] [Info] Number of data points in the train set: 171739, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142623 -> initscore=-1.793670
[LightGBM] [Info] Start training from score -1.793670
Accuracy: 0.7288960125977166

Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      0.72      0.82     60862
         1.0       0.32      0.79      0.46     10262

    accuracy                           0.73     71124
   macro avg       0.64      0.75      0.64     71124
weighted avg       0.86      0.73      0.77     71124


Confusion Matrix:
 [[43754 17108]
 [ 2174  8088]]

AUC Scor

In [ ]:
X_test.shape

(71124, 21)

[LightGBM] [Info] Number of positive: 24494, number of negative: 147245
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.095634 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 203
[LightGBM] [Info] Number of data points in the train set: 171739, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142623 -> initscore=-1.793670
[LightGBM] [Info] Start training from score -1.793670
The prediction was Non Diabetic due to the following features:
- GeneralHealthCondition: 1.1843
- BodyMassIndex: -0.9876
- HighBloodPressure: -0.4433
- AgeBand: 0.3928
- HighCholesterol: 0.3655


The prediction was Diabetic due to the following features:
- HighBloodPressure: 0.6717
- AgeBand: 0.6591
- GeneralHealthCondition: -0.3699
- IncomeLevel: 0.3390
- HadHeartDiseaseorAtack: 0.3104


The prediction was Diabetic due to the following 

/usr/local/lib/python3.10/dist-packages/shap/explainers/_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')


In [ ]:
y_pred[:25]

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1.])

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
"n_estimators": [50, 100, 200],
"learning_rate": [0.01, 0.1, 0.2],
"num_leaves": [31, 63, 127],
"max_depth": [3, 5, 7, 10],
"min_child_samples": [20, 50, 100],
"subsample": [0.8, 1.0],
"colsample_bytree": [0.8, 1.0],
"verbose":[-1]
}

lgbm = LGBMClassifier(random_state=9010, is_unbalance = True)

# Create GridSearchCV instance
grid = GridSearchCV(lgbm, param_grid, cv=5, scoring=["roc_auc","recall"],refit = "roc_auc")

In [ ]:
grid.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
grid.best_estimator_

In [ ]:
print(f"Best parameters: {grid.best_params_}")
print(f"Best score: {grid.best_score_}")

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)


lgbm_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", lgbm_accuracy)
print("")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("")
cf_matrix =  confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nAUC Score:\n", roc_auc_score(y_test, y_pred))

print("\nLightGBM:")

lgbm_recall = recall_score(y_test, y_pred)
lgbm_auc = roc_auc_score(y_test, y_pred)
print(f"AUC: {lgbm_auc}")
print(f"Accuracy score: {lgbm_accuracy}")
print(f"Recall score: {lgbm_recall}")
print(f"F1_score: "+str(f1_score(y_test,y_pred)))